In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import json
import time
from collections import deque
import warnings
from pickle import dump, load
import random
from sklearn.preprocessing import StandardScaler

file_path = '/Users/mymac/Google Drive/My Drive/Forex_Robot/'

In [2]:
def add_fractal(df, i, look_back=3):
    if i >= look_back and i < df.shape[0] - look_back:
        lows = []
        highs = []

        for j in range(1, look_back + 1):
            prev_mid_low, prev_mid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
            future_mid_low, future_mid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

            lows.append(float(prev_mid_low))
            lows.append(float(future_mid_low))
            highs.append(float(prev_mid_high))
            highs.append(float(future_mid_high))

        mid_low, mid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

        if float(mid_low) < min(lows):
            return float(mid_low), 1.0

        elif float(mid_high) > max(highs):
            return float(mid_high), 0.0

        else:
            return np.nan, np.nan

    else:
        return np.nan, np.nan

def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback


def rsi(closes, periods=14):
    close_delta = closes.diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def stoch_rsi(data, k_window=3, d_window=3, window=14):
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()

    stoch = ((data - min_val) / (max_val - min_val)) * 100

    slow_k = stoch.rolling(window=k_window, center=False).mean()

    slow_d = slow_k.rolling(window=d_window, center=False).mean()

    return slow_k, slow_d

def n_macd(macd, macdsignal, lookback=50):
    n_macd = 2 * (((macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min()))) - 1
    n_macdsignal = 2 * (((macdsignal - macdsignal.rolling(lookback).min()) / (macdsignal.rolling(lookback).max() - macdsignal.rolling(lookback).min()))) - 1

    return n_macd, n_macdsignal

def chop(df, lookback=14):
    atr1 = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

def bar_lengths(bar_lens, window=36):
    return bar_lens.rolling(window=window).mean(), bar_lens.rolling(window=window).std()

def sar_lengths(opens, sars, window=36):
    diffs = abs(opens - sars.shift(1))

    return diffs.rolling(window=window).mean(), diffs.rolling(window=window).std()

def supertrend(barsdata, atr_len=3, mult=3):
    curr_atr = atr(barsdata['Mid_High'], barsdata['Mid_Low'], barsdata['Mid_Close'], lookback=atr_len)
    highs, lows = barsdata['Mid_High'], barsdata['Mid_Low']
    hl2 = (highs + lows) / 2
    final_upperband = upper_band = hl2 + mult * curr_atr
    final_lowerband = lower_band = hl2 - mult * curr_atr

    # initialize Supertrend column to True
    supertrend = [True] * len(df)

    close = barsdata['Mid_Close']
    
    for i in range(1, len(df.index)):
        curr, prev = i, i - 1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True

        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False

        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]

            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

    return supertrend, final_upperband, final_lowerband

def heikin_ashi(opens, highs, lows, closes):
    ha_close = list((opens + highs + lows + closes) / 4)
    ha_opens = []

    opens_list, closes_list = list(opens), list(closes)

    for i in range(len(ha_close)):
        if i == 0:
            ha_opens.append((opens_list[i] + closes_list[i]) / 2)

        else:
            ha_opens.append((ha_opens[i - 1] + ha_close[i - 1]) / 2)

    ha_highs = list(pd.DataFrame({'ha_open': ha_opens, 'ha_close': ha_close, 'high': list(highs)}).max(axis=1))
    ha_lows = list(pd.DataFrame({'ha_open': ha_opens, 'ha_close': ha_close, 'low': list(lows)}).min(axis=1))

    return ha_opens, ha_highs, ha_lows, ha_close

def trend_indicator(opens, highs, lows, closes, ema_period=50, smoothing_period=10):
    ha_open, ha_high, ha_low, ha_close = heikin_ashi(opens, highs, lows, closes)

    ha_o_ema = pd.Series.ewm(pd.DataFrame({'ha_open': ha_open}), span=ema_period).mean()
    ha_h_ema = pd.Series.ewm(pd.DataFrame({'ha_high': ha_high}), span=ema_period).mean()
    ha_l_ema = pd.Series.ewm(pd.DataFrame({'ha_low': ha_low}), span=ema_period).mean()
    ha_c_ema = pd.Series.ewm(pd.DataFrame({'ha_close': ha_close}), span=ema_period).mean()

    return pd.Series.ewm(ha_o_ema, span=smoothing_period).mean(), pd.Series.ewm(ha_h_ema, span=smoothing_period).mean(), pd.Series.ewm(ha_l_ema, span=smoothing_period).mean(), pd.Series.ewm(ha_c_ema, span=smoothing_period).mean()

def qqe_mod(barsdata, rsi_period=6, smoothing=5, qqe_factor=5, qqe2_factor=1.61, threshold=3, mult=0.35, sma_length=50):
    wilders_period = rsi_period * 2 - 1

    curr_rsi = rsi(barsdata, periods=rsi_period)
    rsi_ema = pd.Series.ewm(curr_rsi, span=smoothing).mean()
    atr_rsi = abs(rsi_ema.shift(1) - rsi_ema)
    atr_rsi_ema = pd.Series.ewm(atr_rsi, span=wilders_period).mean()
    dar = pd.Series.ewm(atr_rsi_ema, span=wilders_period).mean() * qqe_factor

    newshortband = rsi_ema + dar
    newlongband = rsi_ema - dar

    rsi_ema_list = list(rsi_ema)

    longband = [0]
    for i in range(1, len(rsi_ema_list)):
        if rsi_ema_list[i - 1] > longband[i - 1] and rsi_ema_list[i] > longband[i - 1]:
            longband.append(max(longband[i - 1],newlongband[i]))

        else:
            longband.append(newlongband[i])

    shortband = [0]
    for i in range(1,len(rsi_ema_list)):
        if rsi_ema_list[i - 1] < shortband[i - 1] and rsi_ema_list[i] < shortband[i - 1]:
            shortband.append(min(shortband[i - 1],newshortband[i]))
            
        else:
            shortband.append(newshortband[i])

    longband = pd.Series(longband)
    shortband = pd.Series(shortband)

    trend = np.where(rsi_ema > longband.shift(1), 1, -1)    
    fastatrrsitl = pd.Series(np.where(trend == 1, longband, shortband))

    basis = (fastatrrsitl - 50).rolling(window=sma_length).mean()
    dev = (fastatrrsitl - 50).rolling(window=sma_length).std() * mult
    upper = basis + dev
    lower = basis - dev

    greenbar1 = rsi_ema - 50 > threshold
    greenbar2 = rsi_ema - 50 > upper
    redbar1 = rsi_ema - 50 < threshold
    redbar2 = rsi_ema - 50 < lower

    # uptrend = np.where((greenbar1 & greenbar2), True, False)
    # downtrend = np.where((redbar1 & redbar2), True, False)

    uptrend = np.where((greenbar2), True, False)
    downtrend = np.where((redbar2), True, False)

    return uptrend, downtrend

def williams_r(highs, lows, closes, length=21, ema_length=15):
    highest_highs = highs.rolling(window=length).max()
    lowest_lows = lows.rolling(window=length).min()

    willy = 100 * (closes - highest_highs) / (highest_highs - lowest_lows)
    willy_ema = pd.Series.ewm(willy, span=ema_length).mean()

    return willy, willy_ema

def squeeze(barsdata, length=20, length_kc=20, mult=1.5):
    # Bollinger bands
    m_avg = barsdata['Mid_Close'].rolling(window=length).mean()
    m_std = barsdata['Mid_Close'].rolling(window=length).std(ddof=0)
    upper_bb = m_avg + mult * m_std
    lower_bb = m_avg - mult * m_std

    # Keltner channel
    tr0 = abs(barsdata['Mid_High'] - barsdata['Mid_Low'])
    tr1 = abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    tr2 = abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    tr = pd.concat([tr0, tr1, tr2], axis=1).max(axis=1)
    range_ma = tr.rolling(window=length_kc).mean()
    upper_kc = m_avg + range_ma * mult
    lower_kc = m_avg - range_ma * mult

    # Squeeze
    squeeze_on = (lower_bb > lower_kc) & (upper_bb < upper_kc)

    return squeeze_on

In [3]:
def apply_impact(val, impact):
    return float(val) * float(impact)

def clean_value(val):
    return float(val)
    # if isinstance(val, float) or isinstance(val, int):
    #     return float(val)
    
    # new_val = val.replace(',', '')

    # if 'M' in new_val:
    #     num, _ = new_val.split('M')

    #     return float(num) * 1000000

    # elif 'B' in new_val:
    #     num, _ = new_val.split('B')

    #     return float(num) * 1000000000

    # elif 'k' in new_val:
    #     num, _ = new_val.split('k')

    #     return float(num) * 1000

    # elif '%' in new_val:
    #     num, _ = new_val.split('%')

    #     return float(num) / 100

    # else:
    #     return float(new_val)

def forecast_ratio(forecast, actual):
    if str(forecast) == 'nan' or str(actual) == 'nan':
        return 0.0

    forecast_clean = clean_value(forecast)
    actual_clean = clean_value(actual)
    
    if actual_clean == 0:
        return 0

    return forecast_clean / actual_clean

def previous_ratio(previous, actual):
    if str(previous) == 'nan' or str(actual) == 'nan':
        return 0.0

    previous_clean = clean_value(previous)
    actual_clean = clean_value(actual)
    
    if actual_clean == 0:
        return 0

    return previous_clean / actual_clean

In [4]:
currency_pair = 'Usd_Jpy'
years = '2012-2021'

currency1, currency2 = currency_pair.split('_')
currency1, currency2 = currency1.upper(), currency2.upper()

news = pd.read_csv(file_path + 'events.csv')
news.Date = pd.to_datetime(news.Date, utc=True)
news.drop(news[(news['Impact'] != 'low') & (news['Impact'] != 'med') & (news['Impact'] != 'high')].index, inplace=True)
news.loc[news['Impact'] == 'low', 'Impact'] = 1
news.loc[news['Impact'] == 'med', 'Impact'] = 2
news.loc[news['Impact'] == 'high', 'Impact'] = 3
news['Impact'] = pd.to_numeric(news['Impact'])
# news['Actual_Class'] = news.apply(lambda row: clean_value(row['Actual']), axis=1)
# news['Previous_Class'] = news.apply(lambda row: clean_value(row['Previous']), axis=1)
news['Actual_Class'] = news.apply(lambda row: apply_impact(row['Actual'], row['Impact']), axis=1)
news['Previous_Class'] = news.apply(lambda row: apply_impact(row['Previous'], row['Impact']), axis=1)
news_base = news.loc[news['Currency_Code'] == currency1]
news_counter = news.loc[news['Currency_Code'] == currency2]
news_base.drop(['Currency_Code', 'Actual', 'Previous', 'Actual_Val', 'Forecast_Val', 'Previous_Val'], axis=1, inplace=True)
news_counter.drop(['Currency_Code', 'Actual', 'Previous', 'Actual_Val', 'Forecast_Val', 'Previous_Val'], axis=1, inplace=True)
by_date1 = news_base.groupby('Date')
impact1, actual1, previous1 = by_date1['Impact'].mean().reset_index(), by_date1['Actual_Class'].mean().reset_index(), by_date1['Previous_Class'].mean().reset_index()
news_base = news_base.iloc[0:0]
news_base['Date'], news_base['Impact'], news_base['Actual_Class'], news_base['Previous_Class'] = impact1['Date'], impact1['Impact'], actual1['Actual_Class'], previous1['Previous_Class']
by_date2 = news_counter.groupby('Date')
impact2, actual2, previous2 = by_date2['Impact'].mean().reset_index(), by_date2['Actual_Class'].mean().reset_index(), by_date2['Previous_Class'].mean().reset_index()
news_counter = news_counter.iloc[0:0]
news_counter['Date'], news_counter['Impact'], news_counter['Actual_Class'], news_counter['Previous_Class'] = impact2['Date'], impact2['Impact'], actual2['Actual_Class'], previous2['Previous_Class']

df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_{years}.csv')
df.Date = pd.to_datetime(df.Date, utc=True)
df.reset_index(drop=True, inplace=True)

df['atr'] = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['chop'] = chop(df)
df['vo'] = vo(df['Volume'])

df = pd.merge(df, news_base, how='left', on='Date')
df = pd.merge(df, news_counter, how='left', on='Date')
df.reset_index(drop=True, inplace=True)
df = df.fillna(method='ffill')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

/Users/mymac/miniforge3/envs/forex/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-4-d5bce331ed09>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_base['Date'], news_base['Impact'], news_base['Actual_Class'], news_base['Previous_Class'] = impact1['Date'], impact1['Impact'], actual1['Actual_Class'], previous1['Previous_Class']
<ipython-input-4-d5bce331ed09>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [5]:
df

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,...,atr,adx,chop,vo,Impact_x,Actual_Class_x,Previous_Class_x,Impact_y,Actual_Class_y,Previous_Class_y
0,2018-01-05 00:30:00+00:00,112.755,112.760,112.723,112.733,112.769,112.774,112.737,112.746,112.762,...,0.020714,27.066326,0.507434,0.161919,2.0,-2.0,-2.0,2.0,0.0,0.0
1,2018-01-05 00:35:00+00:00,112.730,112.762,112.730,112.761,112.744,112.775,112.744,112.775,112.737,...,0.021857,26.312560,0.527784,0.111632,2.0,-2.0,-2.0,2.0,0.0,0.0
2,2018-01-05 00:40:00+00:00,112.759,112.766,112.753,112.760,112.773,112.778,112.765,112.774,112.766,...,0.021071,25.513691,0.513912,0.042408,2.0,-2.0,-2.0,2.0,0.0,0.0
3,2018-01-05 00:45:00+00:00,112.762,112.779,112.753,112.779,112.776,112.793,112.767,112.793,112.769,...,0.021929,24.427894,0.529020,0.067168,2.0,-2.0,-2.0,2.0,0.0,0.0
4,2018-01-05 00:50:00+00:00,112.781,112.798,112.757,112.792,112.795,112.811,112.770,112.805,112.788,...,0.023929,23.028395,0.562094,0.152126,2.0,-2.0,-2.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222332,2020-12-31 21:35:00+00:00,103.267,103.269,103.264,103.266,103.283,103.285,103.278,103.279,103.275,...,0.011714,29.865134,0.619210,-0.256577,1.0,0.0,0.0,1.0,0.0,0.0
222333,2020-12-31 21:40:00+00:00,103.265,103.279,103.265,103.279,103.284,103.297,103.281,103.293,103.274,...,0.012286,30.639822,0.582244,-0.239828,1.0,0.0,0.0,1.0,0.0,0.0
222334,2020-12-31 21:45:00+00:00,103.282,103.288,103.275,103.280,103.292,103.305,103.288,103.299,103.287,...,0.012929,31.857399,0.519070,-0.150651,1.0,0.0,0.0,1.0,0.0,0.0
222335,2020-12-31 21:50:00+00:00,103.280,103.286,103.268,103.285,103.296,103.309,103.288,103.300,103.288,...,0.013286,33.351419,0.521246,0.026382,1.0,0.0,0.0,1.0,0.0,0.0


In [8]:
buys_list = []
sells_list = []
nones_list = []

value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]
spread_cutoff = 0.10
risk_reward_ratio = 1.5
each_bar = False
n_bars = 3
pip_movement = 20
use_pullback = True
lookback = n_bars + 1 if use_pullback else n_bars
lookforward = -1 if use_pullback else 0

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

rounding = 3 if 'Jpy' in currency_pair else 5
pip_movement_to_use = pip_movement / 100 if 'Jpy' in currency_pair else pip_movement / 10000
trade, prev_year = None, None

print(f'RUNNING SIMULATION FOR {currency_pair}...')

for i in range(lookback, len(df)):
    curr_date = df.loc[df.index[i], 'Date']

    if prev_year is None or curr_date.year > prev_year:
        prev_year = curr_date.year
        print(prev_year)
        print(f'Buys: {len(buys_list)}')
        print(f'Sells: {len(sells_list)}')
        print(f'Nones: {len(nones_list)}\n')

    curr_bid_open, curr_bid_high, curr_bid_low, curr_ask_open, curr_ask_high, curr_ask_low, curr_mid_open = df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Mid_Open']]
    spread = abs(curr_ask_open - curr_bid_open)

    mid_opens = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Open'])
    mid_highs = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_High'])
    mid_lows = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Low'])
    mid_closes = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Close'])

    if each_bar:
        buy_signal = all([mid_opens[j] < mid_closes[j] and abs(mid_opens[j] - mid_closes[j]) >= pip_movement_to_use for j in range(len(mid_opens))])
        sell_signal = all([mid_opens[j] > mid_closes[j] and abs(mid_opens[j] - mid_closes[j]) >= pip_movement_to_use for j in range(len(mid_opens))])

    else:
        buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement_to_use
        sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement_to_use

    if use_pullback and buy_signal:
        mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
        buy_signal = mid_open1 > mid_close1 and abs(mid_close1 - mid_open1) <= 0.35 * abs(mid_high1 - mid_low1)

    if use_pullback and sell_signal:
        mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
        sell_signal = mid_open1 < mid_close1 and abs(mid_close1 - mid_open1) <= 0.35 * abs(mid_high1 - mid_low1)

    highest_high, lowest_low = max(mid_highs), min(mid_lows)

    if trade is None:
        if buy_signal:
            open_price = float(curr_ask_open)
            stop_loss = round(lowest_low, rounding)

            if stop_loss < open_price:
                curr_pips_to_risk = open_price - stop_loss

                if spread <= curr_pips_to_risk * spread_cutoff:
                    stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), rounding)

                    n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                    trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                    'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                    'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

        elif sell_signal:
            open_price = float(curr_bid_open)
            stop_loss = round(highest_high, rounding)

            if stop_loss > open_price:
                curr_pips_to_risk = stop_loss - open_price

                if spread <= curr_pips_to_risk * spread_cutoff:
                    stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), rounding)

                    n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                    trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                            'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                            'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

    if trade is not None:
        for j in range(i, len(df)):
            curr_date = df.loc[df.index[j], 'Date']
            curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = df.loc[df.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close']]

            if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
                nones_list.append(trade['start_date']) 

                trade = None
                break


            if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
                trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
                day_fees = calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

                if trade_amount + day_fees > 0:
                    buys_list.append(trade['start_date'])

                trade = None
                break

            if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
                nones_list.append(trade['start_date'])

                trade = None
                break

            if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
                trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
                day_fees = calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

                if trade_amount + day_fees > 0:
                    sells_list.append(trade['start_date'])

                trade = None
                break

RUNNING SIMULATION FOR Usd_Jpy...
2018
Buys: 0
Sells: 0
Nones: 0

2019
Buys: 5
Sells: 15
Nones: 32

2020
Buys: 11
Sells: 20
Nones: 52



In [9]:
buy_indices = [(df.index[df['Date'] == curr_date] - 1)[0] for curr_date in buys_list]
sell_indices = [(df.index[df['Date'] == curr_date] - 1)[0] for curr_date in sells_list]
nones_indices = [(df.index[df['Date'] == curr_date] - 1)[0] for curr_date in nones_list]

print(len(buy_indices))
print(len(sell_indices))
print(len(nones_indices))

36
37
116


In [10]:
df.iloc[buy_indices, 13:].mean()

Volume              938.166667
atr                   0.116992
adx                  25.800323
chop                  0.390549
vo                    0.108807
Impact_x              1.674383
Actual_Class_x        0.219136
Previous_Class_x      0.197531
Impact_y              2.013889
Actual_Class_y       -0.361111
Previous_Class_y     -0.111111
dtype: float64

In [11]:
df.iloc[sell_indices, 13:].mean()

Volume              597.027027
atr                   0.091971
adx                  30.631477
chop                  0.341680
vo                    0.148669
Impact_x              2.069369
Actual_Class_x       -0.000901
Previous_Class_x      0.232432
Impact_y              1.898649
Actual_Class_y        0.000000
Previous_Class_y     -0.229730
dtype: float64

In [12]:
df.iloc[nones_indices, 13:].mean()

Volume              796.991379
atr                   0.109191
adx                  29.790361
chop                  0.372419
vo                    0.122157
Impact_x              1.829023
Actual_Class_x        0.178879
Previous_Class_x     -0.046695
Impact_y              1.947557
Actual_Class_y       -0.256466
Previous_Class_y     -0.113218
dtype: float64

In [13]:
currency_pair = 'Eur_Usd'
years = '2021-2022'

currency1, currency2 = currency_pair.split('_')
currency1, currency2 = currency1.upper(), currency2.upper()

news = pd.read_csv(file_path + 'events.csv')
news.Date = pd.to_datetime(news.Date, utc=True)
news.drop(news[(news['Impact'] != 'low') & (news['Impact'] != 'med') & (news['Impact'] != 'high')].index, inplace=True)
news.loc[news['Impact'] == 'low', 'Impact'] = 1
news.loc[news['Impact'] == 'med', 'Impact'] = 2
news.loc[news['Impact'] == 'high', 'Impact'] = 3
news['Impact'] = pd.to_numeric(news['Impact'])
# news['Actual_Class'] = news.apply(lambda row: clean_value(row['Actual']), axis=1)
# news['Previous_Class'] = news.apply(lambda row: clean_value(row['Previous']), axis=1)
news['Actual_Class'] = news.apply(lambda row: apply_impact(row['Actual'], row['Impact']), axis=1)
news['Previous_Class'] = news.apply(lambda row: apply_impact(row['Previous'], row['Impact']), axis=1)
news_base = news.loc[news['Currency_Code'] == currency1]
news_counter = news.loc[news['Currency_Code'] == currency2]
news_base.drop(['Currency_Code', 'Actual', 'Previous', 'Actual_Val', 'Forecast_Val', 'Previous_Val'], axis=1, inplace=True)
news_counter.drop(['Currency_Code', 'Actual', 'Previous', 'Actual_Val', 'Forecast_Val', 'Previous_Val'], axis=1, inplace=True)
by_date1 = news_base.groupby('Date')
impact1, actual1, previous1 = by_date1['Impact'].mean().reset_index(), by_date1['Actual_Class'].mean().reset_index(), by_date1['Previous_Class'].mean().reset_index()
news_base = news_base.iloc[0:0]
news_base['Date'], news_base['Impact'], news_base['Actual_Class'], news_base['Previous_Class'] = impact1['Date'], impact1['Impact'], actual1['Actual_Class'], previous1['Previous_Class']
by_date2 = news_counter.groupby('Date')
impact2, actual2, previous2 = by_date2['Impact'].mean().reset_index(), by_date2['Actual_Class'].mean().reset_index(), by_date2['Previous_Class'].mean().reset_index()
news_counter = news_counter.iloc[0:0]
news_counter['Date'], news_counter['Impact'], news_counter['Actual_Class'], news_counter['Previous_Class'] = impact2['Date'], impact2['Impact'], actual2['Actual_Class'], previous2['Previous_Class']

df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_{years}.csv')
df.Date = pd.to_datetime(df.Date, utc=True)
df.reset_index(drop=True, inplace=True)

df['atr'] = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['chop'] = chop(df)
df['vo'] = vo(df['Volume'])

df = pd.merge(df, news_base, how='left', on='Date')
df = pd.merge(df, news_counter, how='left', on='Date')
df.reset_index(drop=True, inplace=True)
df = df.fillna(method='ffill')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

df_long = pd.read_csv(file_path + f'Oanda_{currency_pair}_H1_{years}.csv')
df_long.Date = pd.to_datetime(df_long.Date, utc=True)
df_long.reset_index(drop=True, inplace=True)

df_long['atr'] = atr(df_long['Mid_High'], df_long['Mid_Low'], df_long['Mid_Close'])
df_long['adx'] = adx(df_long['Mid_High'], df_long['Mid_Low'], df_long['Mid_Close'])
df_long['chop'] = chop(df_long)
df_long['vo'] = vo(df_long['Volume'])

df_long.dropna(inplace=True)
df_long.reset_index(drop=True, inplace=True)

/Users/mymac/miniforge3/envs/forex/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-13-1eb61b1980a7>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_base['Date'], news_base['Impact'], news_base['Actual_Class'], news_base['Previous_Class'] = impact1['Date'], impact1['Impact'], actual1['Actual_Class'], previous1['Previous_Class']
<ipython-input-13-1eb61b1980a7>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [14]:
df

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,...,atr,adx,chop,vo,Impact_x,Actual_Class_x,Previous_Class_x,Impact_y,Actual_Class_y,Previous_Class_y
0,2021-01-04 14:45:00+00:00,1.22991,1.23036,1.22947,1.22979,1.23004,1.23049,1.22961,1.22993,1.22998,...,0.000755,19.146308,0.632758,0.036689,1.0,0.0,0.0,2.0,2.000000,0.000000
1,2021-01-04 14:50:00+00:00,1.22981,1.23004,1.22885,1.22899,1.22993,1.23018,1.22898,1.22914,1.22987,...,0.000803,18.777810,0.642947,0.099424,1.0,0.0,0.0,2.0,2.000000,0.000000
2,2021-01-04 14:55:00+00:00,1.22898,1.22914,1.22830,1.22885,1.22912,1.22928,1.22844,1.22900,1.22905,...,0.000809,17.498369,0.556299,0.098807,1.0,0.0,0.0,2.0,2.000000,0.000000
3,2021-01-04 15:00:00+00:00,1.22886,1.22902,1.22825,1.22827,1.22902,1.22916,1.22838,1.22842,1.22894,...,0.000824,17.054814,0.556067,0.056477,1.0,0.0,0.0,2.0,0.666667,0.666667
4,2021-01-04 15:05:00+00:00,1.22830,1.22861,1.22734,1.22744,1.22841,1.22876,1.22748,1.22756,1.22836,...,0.000852,16.784890,0.457167,0.070977,1.0,0.0,0.0,2.0,0.666667,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74284,2021-12-31 21:35:00+00:00,1.13776,1.13782,1.13774,1.13781,1.13794,1.13800,1.13792,1.13799,1.13785,...,0.000225,21.865993,0.450246,-0.228939,1.0,0.0,0.0,1.0,0.000000,0.000000
74285,2021-12-31 21:40:00+00:00,1.13783,1.13785,1.13771,1.13774,1.13800,1.13802,1.13787,1.13791,1.13792,...,0.000223,21.860638,0.446620,-0.265687,1.0,0.0,0.0,1.0,0.000000,0.000000
74286,2021-12-31 21:45:00+00:00,1.13771,1.13794,1.13758,1.13789,1.13790,1.13818,1.13779,1.13811,1.13780,...,0.000231,21.847559,0.459749,-0.053726,1.0,0.0,0.0,1.0,0.000000,0.000000
74287,2021-12-31 21:50:00+00:00,1.13791,1.13816,1.13761,1.13761,1.13813,1.13838,1.13800,1.13800,1.13802,...,0.000251,21.487700,0.507377,0.117111,1.0,0.0,0.0,1.0,0.000000,0.000000


In [17]:
df_long

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,atr,adx,chop,vo
0,2021-01-04 12:00:00+00:00,1.22925,1.23056,1.22864,1.23028,1.22937,1.23070,1.22878,1.23042,1.22931,1.23062,1.22871,1.23035,5837,0.001505,77.855402,0.390399,0.104962
1,2021-01-04 13:00:00+00:00,1.23030,1.23045,1.22856,1.22982,1.23043,1.23058,1.22870,1.22997,1.23036,1.23052,1.22864,1.22990,7722,0.001484,72.574559,0.426459,0.142061
2,2021-01-04 14:00:00+00:00,1.22984,1.23089,1.22830,1.22885,1.22997,1.23108,1.22844,1.22900,1.22990,1.23098,1.22837,1.22892,9625,0.001590,69.976608,0.439453,0.183780
3,2021-01-04 15:00:00+00:00,1.22886,1.22902,1.22596,1.22712,1.22902,1.22916,1.22613,1.22724,1.22894,1.22908,1.22605,1.22718,10731,0.001694,66.434238,0.471597,0.203536
4,2021-01-04 16:00:00+00:00,1.22708,1.22767,1.22519,1.22532,1.22725,1.22782,1.22532,1.22544,1.22716,1.22774,1.22526,1.22538,9301,0.001806,53.789532,0.496042,0.173497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6217,2021-12-31 17:00:00+00:00,1.13782,1.13857,1.13766,1.13856,1.13797,1.13871,1.13781,1.13870,1.13790,1.13864,1.13774,1.13863,1729,0.001784,17.292895,0.417353,0.090819
6218,2021-12-31 18:00:00+00:00,1.13855,1.13855,1.13741,1.13840,1.13869,1.13869,1.13756,1.13854,1.13862,1.13862,1.13748,1.13847,828,0.001841,19.675722,0.429303,-0.059084
6219,2021-12-31 19:00:00+00:00,1.13842,1.13852,1.13791,1.13820,1.13856,1.13865,1.13805,1.13834,1.13849,1.13858,1.13798,1.13827,619,0.001769,21.590350,0.414306,-0.182146
6220,2021-12-31 20:00:00+00:00,1.13819,1.13845,1.13749,1.13749,1.13832,1.13863,1.13767,1.13767,1.13826,1.13854,1.13758,1.13758,623,0.001785,23.330537,0.417657,-0.268561


In [18]:
rounding = 5 if 'Jpy' not in currency_pair else 3
divider = 10000 if 'Jpy' not in currency_pair else 100
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [21]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, volume_cutoff, chop_cutoff, adx_cutoff, use_news, use_long):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    trade = None
    lookback = n_bars + 1 if use_pullback else n_bars
    lookforward = -1 if use_pullback else 0
    pip_movement /= divider
    pips_to_risk = pips_to_risk / divider if type(pips_to_risk) != str else pips_to_risk

    for i in range(lookback, len(df)):
        curr_bid_open, curr_bid_high, curr_bid_low, curr_ask_open, curr_ask_high, curr_ask_low, curr_mid_open, curr_date = df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Mid_Open', 'Date']]
        
        if use_long and len(df_long.loc[df_long.Date >= curr_date]) == 0:
            break

        curr_long = df_long.loc[df_long.Date <= curr_date]

        if use_long and len(curr_long) < 2:
            continue

        if use_long:
            actual_base, actual_counter = df.loc[df.index[i - 1], ['Actual_Class_x', 'Actual_Class_y']]
            volume, chop, adx = curr_long.loc[curr_long.index[-2], ['vo', 'chop', 'adx']]

        else:
            volume, chop, adx, actual_base, actual_counter = df.loc[df.index[i - 1], ['vo', 'chop', 'adx', 'Actual_Class_x', 'Actual_Class_y']]
        
        spread = abs(curr_ask_open - curr_bid_open)

        mid_opens = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Open'])
        mid_highs = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_High'])
        mid_lows = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Low'])
        mid_closes = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Close'])

        buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement
        sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement

        if use_pullback and buy_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            buy_signal = mid_open1 > mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

        if use_pullback and sell_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            sell_signal = mid_open1 < mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

        volume_signal = volume >= volume_cutoff if volume_cutoff is not None else True
        chop_signal = chop <= chop_cutoff if chop_cutoff is not None else True
        adx_signal = adx >= adx_cutoff if adx_cutoff is not None else True
        news_buy = actual_base > 0 if use_news else True
        news_sell = actual_base < 0 if use_news else True

        buy_signal = buy_signal and volume_signal and chop_signal and adx_signal and news_buy
        sell_signal = sell_signal and volume_signal and chop_signal and adx_signal and news_sell

        highest_high, lowest_low = max(mid_highs), min(mid_lows)

        if trade is None:
            if buy_signal:
                open_price = float(curr_ask_open)

                stop_loss = open_price - pips_to_risk if type(pips_to_risk) != str else lowest_low
                stop_loss = round(stop_loss, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

            elif sell_signal:
                open_price = float(curr_bid_open)
                
                stop_loss = open_price + pips_to_risk if type(pips_to_risk) != str else highest_high
                stop_loss = round(stop_loss, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None    
            closed_trade = True

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [22]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.5]
spread_cutoffs = [0.10]
n_bars_vals = [3]
pip_movement_vals = [20]
use_pullback_vals = [True]
pips_to_risk_vals = ['bars']  # LEAVE THE 'bars' VALUE IN, YOU CAN CHANGE THE OTHER VALUES
pullback_percentages = [0.35, 0.50]
volume_cutoffs = [None, 0.0, 0.10, 0.15, 0.20]
chop_cutoffs = [None, 0.35, 0.50]
adx_cutoffs = [None, 20, 25, 30]
use_news_vals = [True, False]
use_long_vals = [True, False]

n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(n_bars_vals) * len(pip_movement_vals) * len(use_pullback_vals) * len(pips_to_risk_vals) * len(pullback_percentages) * len(volume_cutoffs) * len(chop_cutoffs) * len(adx_cutoffs) * len(use_news_vals) * len(use_long_vals)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for n_bars in n_bars_vals:
            for pip_movement in pip_movement_vals:
                for use_pullback in use_pullback_vals:
                    for pips_to_risk in pips_to_risk_vals:
                        for pullback_percentage in pullback_percentages:
                            for volume_cutoff in volume_cutoffs:
                                for chop_cutoff in chop_cutoffs:
                                    for adx_cutoff in adx_cutoffs:
                                        for use_news in use_news_vals:
                                            for use_long in use_long_vals:
                                                all_combos.append((risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, volume_cutoff, chop_cutoff, adx_cutoff, use_news, use_long))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_n_bars_val = None
best_pip_movement_val = None
best_use_pullback_val = None
best_pips_to_risk = None
best_pullback_percentage = None
best_volume_cutoff = None
best_chop_cutoff = None
best_adx_cutoff = None
best_use_news_val = None
best_use_long_val = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, volume_cutoff, chop_cutoff, adx_cutoff, use_news, use_long in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, volume_cutoff, chop_cutoff, adx_cutoff, use_news, use_long)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_sells))
    print('Num sells: ' + str(n_buys))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'n_bars': n_bars, 'pip_movement': pip_movement, 'use_pullback': use_pullback, 'pips_to_risk': pips_to_risk, 'pullback_percentage': pullback_percentage, 'volume_cutoff': volume_cutoff, 'chop_cutoff': chop_cutoff, 'adx_cutoff': adx_cutoff, 'use_news': use_news, 'use_long': use_long})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_n_bars_val = n_bars
        best_pip_movement_val = pip_movement
        best_use_pullback_val = use_pullback
        best_pips_to_risk = pips_to_risk
        best_pullback_percentage = pullback_percentage
        best_volume_cutoff = volume_cutoff
        best_chop_cutoff = chop_cutoff
        best_adx_cutoff = adx_cutoff
        best_use_news_val = use_news
        best_use_long_val = use_long

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 240

-30.592150000003276
Num buys: 0
Num sells: 3
Num trades: 3
Num wins: 1
Num losses: 2
Win streak: 1
Loss streak: 1
Avg pips risked: 0.002260000000000003
Remaining runs: 239
Best reward so far: -30.592150000003276

136.81773999999237
Num buys: 5
Num sells: 7
Num trades: 12
Num wins: 6
Num losses: 6
Win streak: 3
Loss streak: 3
Avg pips risked: 0.002596666666666673
Remaining runs: 238
Best reward so far: 136.81773999999237

-109.3904699999963
Num buys: 0
Num sells: 2
Num trades: 2
Num wins: 0
Num losses: 2
Win streak: 0
Loss streak: 2
Avg pips risked: 0.002274999999999916
Remaining runs: 237
Best reward so far: 136.81773999999237

-99.9984000000012
Num buys: 2
Num sells: 0
Num trades: 2
Num wins: 0
Num losses: 2
Win streak: 0
Loss streak: 2
Avg pips risked: 0.0024450000000000305
Remaining runs: 236
Best reward so far: 136.81773999999237

0
Num buys: 0
Num sells: 0
Num trades: 0
Num wins: 0
Num losses: 0
Win streak: 0
Loss streak: 0
Remaining runs: 235
Best reward so far: 13

KeyboardInterrupt: 

In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('Best pips to risk: ' + str(best_pips_to_risk))
print('Best pullback percentage: ' + str(best_pullback_percentage))
print('Best volume cutoff: ' + str(best_volume_cutoff))
print('Best chop cutoff: ' + str(best_chop_cutoff))
print('Best adx cutoff: ' + str(best_adx_cutoff))
print('Best use news val: ' + str(best_use_news_val))
print('Best use long val: ' + str(best_use_long_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 766.3267899999956
Best risk/reward ratio: 1.5
Best spread: 0.1
Best n bars val: 3
Best pip movement val: 20
Best use pullback val: True
Best pips to risk: bars
Best pullback percentage: 0.5
Best volume cutoff: 1000
Best chop cutoff: None
Best adx cutoff: None
Best use news val: False
-----------------------
Top results:
{'reward': 593, 'ratio': 1.5, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True, 'pips_to_risk': 'bars', 'pullback_percentage': 0.35, 'volume_cutoff': 800, 'chop_cutoff': 0.5, 'adx_cutoff': None, 'use_news': False}
{'reward': 594, 'ratio': 1.5, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True, 'pips_to_risk': 'bars', 'pullback_percentage': 0.35, 'volume_cutoff': 900, 'chop_cutoff': None, 'adx_cutoff': None, 'use_news': False}
{'reward': 635, 'ratio': 1.5, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True, 'pips_to_risk': 'bars', 'pullback_percentage': 0.35, 'vo